In [1]:
import jax
import jax.numpy as jnp
import jax.tree_util as jtu
import genjax
from genjax.incremental import diff, UnknownChange, NoChange
from genjax.inference.translator import extending_trace_translator
from genjax.inference import smc
from genjax.typing import typecheck
from genjax import ChoiceMap

# Model
@genjax.gen(genjax.Unfold, max_length=10)
@genjax.gen(genjax.Static)
def chain(z_prev):
    z = genjax.normal(z_prev, 1.0) @ "z"
    x = genjax.normal(z, 1.0) @ "x"
    return z


# Proposal and step-indexed translator.
# This function can be freely used inside scan'd code.
def get_translator(t, obs):
    # The proposal signature for this translator is:
    # (obs_chm, prev_particle, *proposal_args)
    @genjax.gen(genjax.Static)
    @typecheck
    def proposal(obs_chm: ChoiceMap, prev_particle: ChoiceMap, *args):
        masked_x = obs_chm[t, "x"]  # from idx choice map
        x = masked_x.unmask()  # will throw if invalid under checkify.
        z = genjax.normal(x, 0.1) @ "z"
        return z

    # The translator allows a bijection on the space of choice maps.
    # We utilize this degree of freedom to change the address representation
    def choice_map_forward(proposal_choices):
        return genjax.indexed_choice_map(t, proposal_choices)

    # The inverse just removes the dynamic index.
    # Unmask is safe under `checkify` - it will throw if
    # the mask is invalid.
    def choice_map_inverse(transformed_choices):
        return transformed_choices[t].unmask()

    translator = extending_trace_translator(
        (diff(t, UnknownChange), diff(0.0, NoChange)),
        proposal,
        (),
        obs,
        choice_map_forward,
        choice_map_inverse,
        # Dynamically checks the bijection, and registers
        # the error with `jax.checkify` if it fails.
        check_bijection=True,
    )
    return translator


# Obs.
obs = genjax.indexed_choice_map(
    [0, 1, 2, 3],
    genjax.choice_map({"x": jnp.array([1.0, 2.0, 3.0, 4.0])}),
)

# SMC with custom proposal.
def extending_smc(key, obs, init_state):
    index_sel = genjax.indexed_select(0)
    obs_slice = obs.slice(0)
    key, sub_key = jax.random.split(key)
    smc_state = smc.smc_initialize(chain, 5).apply(sub_key, obs_slice, (0, init_state))
    obs = jtu.tree_map(lambda v: v[1:], obs)

    def _inner(carry, xs):
        key, smc_state, t = carry
        obs_slice = xs
        t = t + 1
        translator = get_translator(t, obs_slice)
        key, sub_key = jax.random.split(key)
        smc_state = smc.smc_update(translator).apply(
            sub_key,
            smc_state,
        )
        return (key, smc_state, t), (smc_state,)

    (_, final_state, _), (stacked,) = jax.lax.scan(
        _inner,
        (key, smc_state, 0),
        obs,
    )
    return final_state, stacked

In [2]:
from jax.experimental.checkify import checkify

key = jax.random.PRNGKey(314159)

# Use the console to enforce checkify -- this enables
# runtime checks throughout GenJAX.
console = genjax.console(enforce_checkify=True)
with console:
    err, (final_state, stacked) = jax.jit(checkify(extending_smc))(key, obs, 0.0)
    err.throw()

console.print(final_state.particles.get_choices()[3, "z"].unsafe_unmask())

[0.09120648 1.9876677  2.9276989  3.8014348  1.0956165  0.23876664
 1.3302573  1.497613   0.07953135 1.0706444 ]